# Работа с сокетами

Сокеты - это кросс-платформенный механизм для обмена данными между отдельными процессами, которые могут работать на разных машинах. Для взаимодействия по сокетам нужна программа-сервер, слушающая определенный порт и программа-клиент, отправляющая данные этому серверу. 
Общий порядок работы с сокетами в приложении-сервере следующий:
1. Создаем сокет.  
2. Связываем его с IP-адресом машины, на которой работает программа-сервер.  
3. Начинаем прослушивание обращений к заданному порту на обозначенном IP-адресе.  
4. Принимаем обращения и обрабатываем их. Логично делать это в цикле, а не как в коде ниже.  

Рассмотрим простой пример программы-сервера.

In [12]:
%%writefile socket_server.py

import socket

#Создаем сокет
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#Связываем его с IP-адресом машины
sock.bind(("127.0.0.1", 10001))
#Начинаем слушать порт
sock.listen(socket.SOMAXCONN)

#Принимаем обращение. В данном коде сервер может принять только одно обращение.
#Код sock.accept() приостанавливает работу программы в ожидании полученя соединения.
conn, addr = sock.accept()
while True:
    data = conn.recv(1024)
    if not data:
        break
    print(data.decode('utf8'))
    
conn.close()
sock.close

Overwriting socket_server.py


А теперь посмотрим код программы-клиента:

In [2]:
%%writefile socket_client.py

import socket

sock = socket.socket()
sock.connect(("127.0.0.1", 10001))
sock.sendall("ping".encode('utf8'))
sock.close()

#То же, что выше, но более короткая запись
#sock = socket.create_connection(("127.0.0.1", 10001))
#sock.sendall("ping".encode('utf8'))
#sock.close()

Overwriting socket_client.py


Ввиду особенностей Jupyter Notebook, завести это получится только из командной строки. Но решение является вполне работоспособным - стартуем сервер, в другом окне консоли стартуем клиент, смотрим на окно сервера и видим текстовое сообщение. 

Как указано в комментариях к коду сервера, он может принять только одно сообщение, после чего прекратит работу. Очевидно, что это неправильно. Также очевидно, что приведенный выше код не обрабатывает никаких ошибок. Поэтому, с точки зрения безопасности куда выгоднее пользоваться контекстными менеджерами. Перепишем тот же код с исполоьзованием контекстных менеджеров.

In [3]:
%%writefile socket_server_cm.py

import socket

with socket.socket() as sock:
    sock.bind(("", 10001))
    sock.listen()
    
    while True:
        conn, addr = sock.accept()
        with conn:
            while True:
                data = conn.recv(1024)
                if not data:
                    break;
                print(data.decode('utf8'))

Overwriting socket_server_cm.py


In [4]:
%%writefile socket_client_cm.py

import socket
import sys

with socket.create_connection(("127.0.0.1", 10001)) as sock:
    msg = 'ping' if len(sys.argv) == 1 else sys.argv[1]
    sock.sendall(msg.encode('utf8'))

Overwriting socket_client_cm.py


По сути, вышенаписанные клиент и сервер являются telnet-клиентом и telnet-сервером. Из этого следует, что с нашим самописным сервером может работать любой стандартный telnet-клиент. Например, утилита `telnet` из Linux. Для проверки этого достаточно набрать следующий код в командной строке:

```bash
telnet 127.0.0.1 10001
```

# Таймауты и обработка сетевых ошибок

Для начала перепишем наш сервер так, чтобы он мог обрабатывать ошибки. Надо отметить, что приведенный ниже код устанавливает таймаут в 5 секунд от момента установки соединения с клиентом до момента получения от него данных. При отсутствии такого таймаута сервер мог бы ждать данных бесконечно долго, а при установки его в 0 ждал бы тоже, но в неблокирующем режиме (о таком режиме поговорим чуть позднее). В нашем случае, когда таймаут явно установлен, если по его истечении не поступит никаких команд с клиента, то сервер сгенерирует исключение `socket.timeout`.

In [13]:
%%writefile socket_server_eh.py

import socket
import sys

def run_server(port_num: int, batch_size: int = 1024, encoding: str = 'utf8'):
    with socket.socket() as sock:
        sock.bind(("", port_num))
        sock.listen()
        print(f'Listening to port {port_num}...')
        while True:
            conn, addr = sock.accept()
            print(f'Connection {conn} is accepted at address {addr}.')
            conn.settimeout(5)
            with conn:
                while True:
                    try:
                        data = conn.recv(batch_size)
                    except socket.timeout:
                        print('Close connection by timeout')
                        break
                    if not data:
                        break
                    print(data.decode(encoding))
                print(f'Connection {conn} is processed and closed.')

def main(*args):
    try:
        port_num = 10001 if len(args) == 0 else int(args[0])
        run_server(port_num)
    except Exception as e:
        print(e)

if __name__ == '__main__':
    main(*sys.argv[1:])

Overwriting socket_server_eh.py


А теперь напишем клиента с обработкой ошибок.

In [16]:
%%writefile socket_client_eh.py

import socket
import sys

def send_message(msg: str, host: str = '127.0.0.1', port: int = 10001, connect_timeout: int = 5, socket_read_timeout: int = 2):
    with socket.create_connection((host, port), connect_timeout) as sock:
        sock.settimeout(socket_read_timeout)
        try:
            sock.sendall(msg.encode('utf8'))
        except socket.timeout:
            print('send data timeout')
        except socket.error as ex:
            print(f'send data error: {ex}')

def main(*args):
    msg = 'ping' if len(args) == 0 else args[0]
    send_message(msg)

if __name__ == '__main__':
    main(*sys.argv[1:])

Overwriting socket_client_eh.py


В коде выше мы задали два таймаута: один для установки соединения (5 сек.) и один для отправки сообщения (2 сек.). Из этого следует, что по-хорошему мы в случае ошибки установки соединения по таймауту должны попробовать подключиться еще сколько-то раз, а при ошибке отправки данных, должны попытаться сделать это еще сколько-то раз.

# Одновременная работа с несколькими соединениями

Сначала посмотрим на созданный ранее код простого сервера и поймем, почему он непригоден для работы с несколькими соединениями.

In [17]:
%%writefile socket_server_cm.py

import socket

with socket.socket() as sock:
    sock.bind(("", 10001))
    sock.listen()
    
    while True:
        conn, addr = sock.accept()
        with conn:
            while True:
                data = conn.recv(1024)
                if not data:
                    break;
                print(data.decode('utf8'))

Overwriting socket_server_cm.py


А проблема здесь в том, что мы слушаем соединения и обрабатываем их (начиная с метода `sock.accept()`) в одном и том же потоке. То есть, пока мы обрабатываем одно соединение, все остальные клиенты вынуждены ждать окончания его обработки. Самым очевидным решением данного вопроса является создание сразу после метода `sock.accept()` отдельного процесса или потока, обрабатывающего запрос со стороны клиента. При этом мы помним, что каждый процесс в операционной системе имеет свою память, которую он не делит с другими процессами и каждым процессом операционная система управляет отдельно. То есть, если к нашему серверу будет обращаться 10000 клиентских приложений, то нам придется создать 10000 процессов, что явно усложнит работу операционной системы. Поэтому, если речь идет о каких-то несложных задачах, то лучше создавать потоки.

Посмотрим пример обработки сетевых запросов в несколько потоков.

In [21]:
%%writefile socket_server_th.py

import socket
import threading
import sys

def process_request(conn, addr):
    print(f'connected client: {addr}')
    with conn:
        while True:
            data = conn.recv(1024)
            if not data:
                break;
            print(data.decode('utf8'))

def run_server(port_num: int = 10001):
    with socket.socket() as sock:
        sock.bind(("", 10001))
        sock.listen()
        
        while True:
            print('server is waiting for request...')
            conn, addr = sock.accept()
            th = threading.Thread(target=process_request, args=(conn, addr))
            th.start()
        
def main(*args):
    try:
        port_num = 10001 if len(args) == 0 else int(args[0])
        run_server(port_num)
    except Exception as e:
        print(e)

if __name__ == '__main__':
    main(*sys.argv[1:])

Overwriting socket_server_th.py


Если очень хочется реализовать одновременную обработку соединений не на потоках, а на процессах, то это будет выглядеть вот так:

In [25]:
%%writefile socket_server_pr.py

import sys
import socket
import multiprocessing
import threading
import os

def process_request(conn, addr):
    print(f'connected client: {addr}')
    with conn:
        while True:
            data = conn.recv(1024)
            if not data:
                break;
            print(data.decode('utf8'))
            
def worker(sock):
    while True:
        print(f'server (pid={os.getpid()}) is waiting for request...')
        conn, addr = sock.accept()
        th = threading.Thread(target=process_request, args=(conn, addr))
        th.start()

def run_server(port_num: int = 10001, worker_count: int = 4):
    with socket.socket() as sock:
        sock.bind(("", 10001))
        sock.listen()
        workers = [multiprocessing.Process(target=worker, args=(sock,)) for _ in range(worker_count)]
        for w in workers:
            w.start()
        for w in workers:
            w.join()

def main(*args):
    try:
        port_num = 10001 if len(args) == 0 else int(args[0])
        run_server(port_num)
    except Exception as e:
        print(e)

if __name__ == '__main__':
    main(*sys.argv[1:])

Overwriting socket_server_pr.py


# Асинхронное программирование

